CIFAR10 notebook from RNA (CPV rework) - Luis Serrano Hernández

Install Keras just in case...

In [ ]:
!pip3 install keras

Imports...

In [ ]:
from __future__ import print_function

from keras.callbacks import LearningRateScheduler as LRS
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Reshape, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import GaussianNoise as GN
from keras.layers.normalization import BatchNormalization as BN
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

Define batch size, number of epochs and number of classes

In [ ]:
batch_size = 128
epochs = 75
num_classes=10

Load CIFAR10 and transform

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


print(x_train.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

(50000, 32, 32, 3)
50000 train samples
10000 test samples


Data Augmentation with an ImageGenerator


In [ ]:
## Data Augmentation with an ImageGenerator
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.3,
    rotation_range=45,
    vertical_flip=True)

Define a convolutional block with Batch Normalization, Gaussian Noise and MaxPool

In [ ]:
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0):
  if (ishape!=0):
    model.add(Conv2D(filters, (3, 3), padding='same',
                 input_shape=ishape))
  else:
    model.add(Conv2D(filters, (3, 3), padding='same'))
  
  model.add(BN())
  model.add(GN(0.3))
  model.add(Activation('relu'))
  #model.add(Dropout(0.25))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  return model

Define the NN topology, a sequential model with 2 hidden layers

In [ ]:
## DEF NN TOPOLOGY  
model = Sequential()

model=CBGN(model,32,x_train.shape[1:])
model=CBGN(model,64)
model=CBGN(model,128)
model=CBGN(model,256)
model=CBGN(model,512)

model.add(Flatten())
model.add(Dense(512))
model.add(BN())
model.add(GN(0.3))
model.add(Dropout(0.25))
model.add(Activation('relu'))

"""
model.add(Dense(1024))
model.add(BN())
model.add(GN(0.2))
model.add(Dropout(0.35))
model.add(Activation('relu'))
"""

model.add(Dense(num_classes))
model.add(Activation('relu'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise (GaussianNois (None, 32, 32, 32)        0         
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        2

Define an optimizer 

In [ ]:
sgd=SGD(lr=0.1, decay=0.0, momentum=0.0)

def scheduler(epoch):
  if epoch < epochs / 3:
    return 0.1
  elif epoch < epochs / 3 * 2:
    return 0.05
  else:
    return 0.02

# Callbaks
set_lr = LRS(scheduler)
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Compile the model, define loss and link the optimizer



In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

Finally, train the model and evaluate over the test data

In [ ]:
history=model.fit(datagen.flow(x_train, y_train,batch_size=batch_size),
                  steps_per_epoch=len(x_train) / batch_size, 
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  callbacks=[set_lr, es],
                  verbose=1)


score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/150
390/390 [==============================] - 69s 63ms/step - loss: 2.0271 - accuracy: 0.2953 - val_loss: 2.1858 - val_accuracy: 0.2666
Epoch 2/150
390/390 [==============================] - 23s 59ms/step - loss: 1.6383 - accuracy: 0.4072 - val_loss: 1.4223 - val_accuracy: 0.4854
Epoch 3/150
390/390 [==============================] - 23s 59ms/step - loss: 1.5230 - accuracy: 0.4510 - val_loss: 1.3924 - val_accuracy: 0.5056
Epoch 4/150
390/390 [==============================] - 23s 59ms/step - loss: 1.4647 - accuracy: 0.4747 - val_loss: 1.7247 - val_accuracy: 0.4274
Epoch 5/150
390/390 [==============================] - 23s 60ms/step - loss: 1.4075 - accuracy: 0.4924 - val_loss: 1.2633 - val_accuracy: 0.5519
Epoch 6/150
390/390 [==============================] - 23s 59ms/step - loss: 1.3507 - accuracy: 0.5134 - val_loss: 1.1424 - val_accuracy: 0.5951
Epoch 7/150
390/390 [==============================] - 23s 59ms/step - loss: 1.3133 - accuracy: 0.5306 - val_loss: 1.1977 - val_ac

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()